In [ ]:
!pip install -q PyGithub torch transformers accelerate bitsandbytes

import os
from github import Github
import torch
from transformers import pipeline
import getpass
from huggingface_hub import login

HF_TOKEN = getpass.getpass("Cole seu token da Hugging Face: ")
login(HF_TOKEN)

# Repositório alvo
REPO_NAME = "harry0703/MoneyPrinterTurbo"

# --- 1. EXTRAÇÃO DE DADOS DO GITHUB (ACESSO ANÔNIMO) ---
def get_repo_context(repo_name):
    print(f"🔍 Coletando dados de: {repo_name} (Modo Público)...")

    # Inicializa sem token (limite de 60 requisições/hora por IP)
    g = Github()
    repo = g.get_repo(repo_name)

    # Coleta branches (focando nas ativas)
    branches = [b.name for b in repo.get_branches()][:10]

    # Coleta releases/tags (para identificar versionamento)
    tags = [t.name for t in repo.get_tags()][:10]

    # Coleta commits recentes da branch default para ver o padrão de mensagens
    default_branch = repo.default_branch
    commits = repo.get_commits(sha=default_branch)[:10]

    # Formata mensagens de commit
    commit_msgs = [f"[{c.author.login if c.author else 'unknown'}] {c.commit.message.splitlines()[0]}" for c in commits]

    context_text = f"""
    ANALISE ESTE REPOSITÓRIO:
    Nome: {repo_name}
    Branch Principal (Default): {default_branch}
    Branches Ativas no Repo Original: {', '.join(branches)}
    Últimas Tags (Releases): {', '.join(tags)}

    Últimos 10 Commits (Autor + Mensagem):
    {chr(10).join(commit_msgs)}
    """
    return context_text

# --- 2. ANÁLISE COM LLAMA 3.2 ---
def analyze_with_llama(repo_context, hf_token):
    print("🦙 Iniciando análise com Llama-3.2-3B-Instruct...")

    model_id = "meta-llama/Llama-3.2-3B-Instruct"

    pipe = pipeline(
        "text-generation",
        model=model_id,
        token=hf_token, # Obrigatório para modelos Llama oficiais
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )

    prompt = [
        {"role": "system", "content": "Você é um Tech Lead especialista em Open Source. Analise os metadados do git e descreva o fluxo de trabalho."},
        {"role": "user", "content": f"""

        Dados:
        {repo_context}

         1. Modelo de Fluxo de Trabalho (Branching Model)

        Identifique qual modelo está sendo utilizado (ex: Git Flow, GitHub Flow, Trunk-Based, híbrido).

        Justifique com evidências observáveis (branches, merges, duração dos branches).

        2. Estratégia de Release

        Identifique qual estratégia de release o projeto utiliza
        (ex: time-based, feature-based, continuous delivery/deployment).

        Analise o padrão de versionamento adotado
        (ex: Semantic Versioning ou variação).

        """}
    ]

    outputs = pipe(
        prompt,
        max_new_tokens=600,
        do_sample=True,
        temperature=0.5,
    )

    return outputs[0]["generated_text"][-1]["content"]

# --- EXECUÇÃO ---
if __name__ == "__main__":
    if torch.cuda.is_available():
        print(f"🚀 Rodando em GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️ Rodando em CPU (pode ser lento).")

    try:
        # Chamada simplificada sem token do GitHub
        context = get_repo_context(REPO_NAME)

        analise = analyze_with_llama(context, HF_TOKEN)

        print("\n" + "="*50)
        print(f"📊 RELATÓRIO: {REPO_NAME}")
        print("="*50)
        print(analise)

    except Exception as e:
        print(f"❌ Erro: {e}")

Cole seu token da Hugging Face: ··········
🚀 Rodando em GPU: Tesla T4
🔍 Coletando dados de: harry0703/MoneyPrinterTurbo (Modo Público)...
🦙 Iniciando análise com Llama-3.2-3B-Instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



📊 RELATÓRIO: harry0703/MoneyPrinterTurbo
**Análise do Fluxo de Trabalho do Repositório Git**

**1. Modelo de Fluxo de Trabalho**

O modelo de fluxo de trabalho utilizado no repositório Git é o **GitHub Flow**. Isso é evidenciado pelo seguinte:

*   Existem branches ativas no repo original, como `dev` e `main`, o que indica que o projeto utiliza uma abordagem de desenvolvimento contínuo.
*   As tags de release são frequentes e estão sendo utilizadas para marcar pontos importantes no ciclo de vida do projeto, como a versão 1.2.6.
*   Os commits estão sendo feitos regularmente e estão sendo integrados em branches ativas, o que indica uma abordagem de desenvolvimento contínuo.

**2. Estratégia de Release**

A estratégia de release utilizada no projeto é a **feature-based**, ou seja, as novas funcionalidades são desenvolvidas em branches separados e, posteriormente, são integradas na branch principal (`main`) após a aprovação.

Isso é evidenciado pelo seguinte:

*   As tags de release estã